In [1]:
import pandas as pd
import numpy as np
import math
import urllib
import requests
import asyncio
from bs4 import BeautifulSoup as BS

In [ ]:
#async functions

def snag_shit_pandas(url):
    response = pd.read_html(url)
    return response

async def snag_shit_soups(url):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    response = soup.find_all('a') 
    return await response


#asynchronous  info
# Get a reference to the event loop
loop = asyncio.get_event_loop()

# Create the list of shit we want to get asnyc
requests = [asyncio.ensure_future(snag_shit_pandas(someDumbassURL)),
            asyncio.ensure_future(snag_shit_pandas(someOtherDumbShitURL))]

# Run the requests
responses = loop.run_until_complete(asyncio.gather(*requests))

responses

#found this dope shit too. add it if ou want to quit long stuff
try:
    loop.run_forever()
except KeyboardInterrupt:
    # Canceling pending tasks and stopping the loop
    asyncio.gather(*asyncio.Task.all_tasks()).cancel()
    # Stopping the loop
    loop.stop()
    # Received Ctrl+C
    loop.close() 

In [ ]:
#--------

In [2]:
#inputs stuff

tickers = ['OPHT', 'MNTA']
#tickers = ['MNTA', 'AON', 'OPHT', 'IIVI', 'MMM', 'GE']
filing_input_type = 'DEF+14'
table_var = "Summary Compensation Table"
period = 2

In [3]:
url_list = []

for i in range(0, len(tickers)):
    url_list.append('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers[i] + '&owner=include&action=getcompany&type=' + filing_input_type)    
url_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=OPHT&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=MNTA&owner=include&action=getcompany&type=DEF+14']

In [4]:
#BS grabs links here
tag_list = []
document_links = []

for j in range(0,len(url_list)):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    doc_link = soup.find_all('a')
    count = 0

    for i in range(0,len(doc_link)):
        x = doc_link[i].get('href')
        y = x.split('/')
        if len(y) > 1:
            if (y[1] == 'Archives' and count < period):
                tag_list.append([tickers[j],x])
                count = count + 1

    for k in range(0,len(tag_list)):
        index_page = requests.get('https://www.sec.gov'+tag_list[k][1])
        soup2= BS(index_page.content, 'lxml')
        doc_link2 = soup2.find_all('a')
        if tag_list[k][0] == tickers[j]:
            document_links.append([tickers[j],doc_link2[8].get('href')])
    
document_links

[['OPHT',
  '/Archives/edgar/data/1410939/000141093918000020/proxystatementfor2018annua.htm'],
 ['OPHT',
  '/Archives/edgar/data/1410939/000104746917002856/a2231921zdef14a.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm']]

In [5]:
filing_reader = []
for i in range(0,len(document_links)):
    url = 'https://www.sec.gov'+document_links[i][1]
    filing_reader.append([document_links[i][0],pd.read_html(url)])

In [55]:
filing_reader
#filing_reader[2][1][21]

[['OPHT', [                                              0  \
   0                                           NaN   
   1                                           NaN   
   2                     Filed by the Registrant x   
   3  Filed by a Party other than the Registrant o   
   4                    Check the appropriate box:   
   5                                             o   
   6                                             o   
   7                                             x   
   8                                             o   
   9                                             o   
   
                                                      1  
   0                                                NaN  
   1                                                NaN  
   2                                                NaN  
   3                                                NaN  
   4                                                NaN  
   5                        Preliminary Proxy 

In [56]:
len(filing_reader)

4

In [97]:
#let's find the right table

totes_scores = []
filings_scores = []
table_location = []


#does it make sense to include "officer" ?
searchParameters = ("Name|Officer|Position|Salary|Stock|Award|Option|Non‑Equity Incentive Plan|Compensation|All Other|Total|Year")

for k in range(0, len(filing_reader)):
    for j in range(0,len(filing_reader[k][1])):
        interim = filing_reader[k][1][j].dropna(how = 'all', axis = 1).dropna(how = 'all', axis = 0).reset_index(drop=True)
        interim.columns = range(interim.shape[1])
        scorer = []

        for i in range(0,interim.shape[0]):
            scorer.append(interim.iloc[i].str.contains(searchParameters).sum())

        totes_scores.append(sum(scorer))
    
    location = totes_scores.index(max(totes_scores))
    table_location.append(location)
    totes_scores = []






In [98]:
table_location
#should be 20 for second slot, first slot also wrong

[4, 4, 21, 128]

In [103]:
filing_reader[1][1][4].dropna(how = 'all', axis = 1).dropna(how = 'all', axis = 0).reset_index(drop=True)

,0,2,3
0,NaN,Page,NaN
1,Information About the Annual Meeting and Voting,NaN,1.0
2,Votes Required,NaN,2.0
3,CORPORATE GOVERNANCE,NaN,4.0
4,Board of Directors,NaN,4.0
5,How Our Board Is Organized,NaN,7.0
6,Board Committees,NaN,8.0
7,Compensation Committee Interlocks and Insider ...,NaN,10.0
8,Board Meetings and Attendance,NaN,10.0
9,Board Processes,NaN,10.0


In [113]:
nxt = filing_reader[1][1][4].dropna(how = 'all', axis = 1).dropna(how = 'all', axis = 0).reset_index(drop=True)
nxt.columns = range(nxt.shape[1])

for i in range(0,nxt.shape[0]):
    scorer.append(nxt.iloc[i].str.contains(searchParameters).sum())
sum(scorer)

66

In [112]:
nxt.iloc[1].str.contains(searchParameters).sum()

0

In [84]:
#--- Everything works up to here 